https://www.kaggle.com/berkayalan/vaccinations-in-united-states
us_vaccinations.csv

# The main goal of this dashboard is to display the pogress of vaccinations in the 52 United states

## in this notebook I will eplore the best way to display the data. Later I will make a django app where I am aiming to display the following views:
* A heatmap of the united states to keep track of the amount of vacinnations in each state
* A linechart for each state filtered by state and date to diaplay the change in vaccinations
* A bar chart of pie chart to display the amount of people vaccinted compared to the total population per state

*Seaborn: Size of the state == box --> vaccination vs total population filling up the box

# Imports

In [99]:
import pandas as pd
import sqlite3 as sql
from bokeh.io import push_notebook, output_notebook 
from bokeh.plotting import figure,show
from bokeh.embed import components
from bokeh.models import ColumnDataSource, HoverTool,Legend
from bokeh.palettes import Plasma
from bokeh.transform import factor_cmap

In [9]:
output_notebook() #to see notebook in the app

Loading BokehJS ...

## Opening dataset that contains vaccine information per state

In [2]:
db = '../dash/project_2.db'
conn = sql.connect(db)

In [4]:
vacc = pd.read_sql("SELECT * from vacc", conn, index_col='index')
vacc.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
index,,,,,,,,,,,,,,
0,12.01.2021,Alabama,78134.0,377025.0,70861.0,0.15,21551.0,7270.0,16438.0,25385.0,0.0,0.0,0.0,0.207
1,13.01.2021,Alabama,84040.0,378975.0,74792.0,0.19,25934.0,9245.0,19360.0,26846.0,5906.0,5906.0,1205.0,0.222
2,14.01.2021,Alabama,92300.0,435350.0,80480.0,0.19,32143.0,9245.0,23377.0,32356.0,8260.0,7083.0,1445.0,0.212
3,15.01.2021,Alabama,100567.0,444650.0,86956.0,0.28,44318.0,13488.0,28126.0,44386.0,8267.0,7478.0,1525.0,0.226
4,16.01.2021,Alabama,100567.0,444650.0,86956.0,0.28,44318.0,13488.0,28126.0,44386.0,7557.0,7498.0,1529.0,0.226


In [86]:
total = pd.read_sql("SELECT * from total", conn, index_col='index')
total.head()

,state,latitude,longitude,location,state_population,daily_vaccinations,doses_needed,perc_daily_vacc_pop
index,,,,,,,,
0,AK,63.588753,-154.493062,Alaska,733391,601522,1466782,41
1,AL,32.318231,-86.902298,Alabama,5024279,3010720,10048558,29
2,AR,35.20105,-91.831833,Arkansas,3011524,2188280,6023048,36
3,AZ,34.048928,-111.093731,Arizona,7151502,6047164,14303004,42
4,CA,36.778261,-119.417932,California,39538223,39330806,79076446,49


In [88]:
total = total.sort_values('perc_daily_vacc_pop',ascending=False)
total

,state,latitude,longitude,location,state_population,daily_vaccinations,doses_needed,perc_daily_vacc_pop
index,,,,,,,,
47,VT,44.558803,-72.577841,Vermont,643077,787136,1286154,61
19,MA,42.407211,-71.382437,Massachusetts,7029917,8229843,14059834,58
7,DC,38.905985,-77.033418,District of Columbia,689545,788817,1379090,57
6,CT,41.603221,-73.087749,Connecticut,3605944,4048765,7211888,56
21,ME,45.253783,-69.445469,Maine,1362359,1484927,2724718,54
11,HI,19.898682,-155.665857,Hawaii,1455271,1600159,2910542,54
40,RI,41.580095,-71.477429,Rhode Island,1097379,1169226,2194758,53
30,NH,43.193852,-71.572395,New Hampshire,1377529,1417612,2755058,51
20,MD,39.045755,-76.641271,Maryland,6177224,6289805,12354448,50


In [148]:
select_top = 10
select_bottom = select_top

In [149]:
grouped = total.groupby('location')['perc_daily_vacc_pop'].sum().sort_values(ascending=False).to_frame().iloc[:select_top]

In [150]:
grouped

,perc_daily_vacc_pop
location,
Vermont,61
Massachusetts,58
District of Columbia,57
Connecticut,56
Hawaii,54
Maine,54
Rhode Island,53
New Hampshire,51
New Jersey,50


In [151]:
top_total = total[:select_top]
bottom_total = total[-select_bottom:]

In [152]:
bottom_total

,state,latitude,longitude,location,state_population,daily_vaccinations,doses_needed,perc_daily_vacc_pop
index,,,,,,,,
36,OK,35.007752,-97.092877,Oklahoma,3959353,2895952,7918706,36
10,GA,32.157435,-82.907123,Georgia,10711908,7828449,21423816,36
2,AR,35.20105,-91.831833,Arkansas,3011524,2188280,6023048,36
50,WV,38.597626,-80.454903,West Virginia,1793716,1268476,3587432,35
43,TN,35.517491,-86.580447,Tennessee,6910840,4746916,13821680,34
51,WY,43.075968,-107.290284,Wyoming,576851,393166,1153702,34
13,ID,44.068202,-114.742041,Idaho,1839106,1238335,3678212,33
18,LA,31.244823,-92.145024,Louisiana,4657757,3071329,9315514,32
25,MS,32.354668,-89.398528,Mississippi,2961279,1787175,5922558,30


In [153]:
top_total

,state,latitude,longitude,location,state_population,daily_vaccinations,doses_needed,perc_daily_vacc_pop
index,,,,,,,,
47,VT,44.558803,-72.577841,Vermont,643077,787136,1286154,61
19,MA,42.407211,-71.382437,Massachusetts,7029917,8229843,14059834,58
7,DC,38.905985,-77.033418,District of Columbia,689545,788817,1379090,57
6,CT,41.603221,-73.087749,Connecticut,3605944,4048765,7211888,56
21,ME,45.253783,-69.445469,Maine,1362359,1484927,2724718,54
11,HI,19.898682,-155.665857,Hawaii,1455271,1600159,2910542,54
40,RI,41.580095,-71.477429,Rhode Island,1097379,1169226,2194758,53
30,NH,43.193852,-71.572395,New Hampshire,1377529,1417612,2755058,51
20,MD,39.045755,-76.641271,Maryland,6177224,6289805,12354448,50


In [154]:
source = ColumnDataSource(top_total)

In [155]:
country_names = source.data['location'].tolist()

In [156]:
colors = factor_cmap('location', palette=Plasma[select_top], factors=country_names)

In [157]:
#p = figure(x_range=country_values, title= f'top {select_top} countries with highest total precentage of population vaccinated')

In [158]:
# p.vbar(x='location', top='perc_daily_vacc_pop', source=source, width=0.8, color=colors)
# p.add_tools(HoverTool())
# p.xaxis.axis_label = 'Total Vaccinated (%)'
# p.yaxis.axis_label = 'State'


# show(p)

In [159]:
type(country_values)

list

In [160]:
p = figure(y_range=country_values, title= f'Top {select_top} countries with highest total precentage of population vaccinated')
p.hbar(y='location', right='perc_daily_vacc_pop', source=source, height=0.8, color=colors)
p.add_tools(HoverTool())
p.xaxis.axis_label = 'Total Population Vaccinated (%)'
p.yaxis.axis_label = 'State'

show(p)

In [55]:
color_map=factor_cmap(field_name='country_values',palette=Spectral6, factors=country_values)

/Users/clarizamayo/opt/anaconda3/lib/python3.8/site-packages/bokeh/models/mappers.py:147: UserWarning: Palette length does not match number of factors. ['Rhode Island', 'New Hampshire', 'New Jersey', 'New Mexico'] will be assigned to `nan_color` gray
  warnings.warn("Palette length does not match number of factors. %s will be assigned to `nan_color` %s" % (extra_factors, self.nan_color))


In [126]:
# p.vbar(x=country_values, top='perc_daily_vacc_pop', source=source, width=0.8, color=color_map, legend_label='Amount of Fans')
# p.add_tools(HoverTool())

# p.title.text ="World's Most Popular Heavy Metal Bands"
# p.xaxis.axis_label = 'Bands'
# p.yaxis.axis_label = 'Number of Fans'

# show(p)

In [127]:
# def top_countries():
#     source = ColumnDataSource(data=top_total)
#     print(source.data('perc_daily_vacc_pop'))
# #     p = figure(x_range=source.data['Country'], title="Number of Bombings per Country")
# #     p.hbar(y=new_df['Country'], right=new_df['Counts'], height=0.8, color=Spectral6, legend_label='label')
# #     p.add_tools(HoverTool())
# #     p.xaxis.axis_label = 'Number of Bombings'
# #     p.yaxis.axis_label = 'Country'
# #     return p

# top_countries()

In [20]:
select_bottom = 10

In [29]:
bottom_total = total[-select_bottom:]
bottom_total

,state,latitude,longitude,location,state_population,daily_vaccinations,doses_needed,perc_daily_vacc_pop
index,,,,,,,,
36,OK,35.007752,-97.092877,Oklahoma,3959353,2895952,7918706,36
10,GA,32.157435,-82.907123,Georgia,10711908,7828449,21423816,36
2,AR,35.20105,-91.831833,Arkansas,3011524,2188280,6023048,36
50,WV,38.597626,-80.454903,West Virginia,1793716,1268476,3587432,35
43,TN,35.517491,-86.580447,Tennessee,6910840,4746916,13821680,34
51,WY,43.075968,-107.290284,Wyoming,576851,393166,1153702,34
13,ID,44.068202,-114.742041,Idaho,1839106,1238335,3678212,33
18,LA,31.244823,-92.145024,Louisiana,4657757,3071329,9315514,32
25,MS,32.354668,-89.398528,Mississippi,2961279,1787175,5922558,30
